# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in Torch / NamedTensor:

1. A count-based trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models.


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [1]:
#!pip install -q torch torchtext opt_einsum
#!pip install -qU git+https://github.com/harvardnlp/namedtensor

In [1]:
import torch
import torchtext
from torchtext.vocab import Vectors

from namedtensor import ntorch, NamedTensor
from namedtensor.text import NamedField

from load import load_text
from models import TrigramModel
from collections import Counter
import numpy as np

In [2]:
train_iter, val_iter, test_iter, TEXT = load_text()

In [3]:
model = TrigramModel(.8, .19, len(TEXT.vocab))
model.fit(train_iter)

In [4]:
model.predict(next(iter(val_iter))).shape

OrderedDict([('batch', 10), ('seqlen', 32), ('distribution', 10001)])

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [3]:
# Our input $x$
TEXT = NamedField(names=("seqlen",))

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [4]:
#!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/input.txt
#!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.5k.txt
#!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.txt
#!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/valid.txt

In [5]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [6]:
print('len(train)', len(train))

len(train) 1


Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [7]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

len(TEXT.vocab) 10001


When debugging you may want to use a smaller vocab size. This will run much faster.

In [8]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [9]:
from torchtext.data.iterator import BPTTIterator
from torchtext.data import Batch, Dataset
import math
 

class NamedBpttIterator(BPTTIterator):
    def __iter__(self):
        text = self.dataset[0].text
        TEXT = self.dataset.fields['text']
        TEXT.eos_token = None
        text = text + ([TEXT.pad_token] * int(math.ceil(len(text) / self.batch_size)
                                              * self.batch_size - len(text)))
        data = TEXT.numericalize(
            [text], device=self.device)
        data = (data
            .stack(("seqlen", "batch"), "flat")
            .split("flat", ("batch", "seqlen"), batch=self.batch_size)
            .transpose("seqlen", "batch")
        )

        dataset = Dataset(examples=self.dataset.examples, fields=[
            ('text', TEXT), ('target', TEXT)])
        while True:
            for i in range(0, len(self) * self.bptt_len, self.bptt_len):
                self.iterations += 1
                seq_len = min(self.bptt_len, len(data) - i - 1)
                yield Batch.fromvars(
                    dataset, self.batch_size,
                    text = data.narrow("seqlen", i, seq_len),
                    target = data.narrow("seqlen", i+1, seq_len),
                )
                         
            if not self.repeat:
                return

In [12]:
train_iter, val_iter, test_iter = NamedBpttIterator.splits(
    (train, val, test), batch_size=10, device=torch.device("cpu"), bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [25]:
it = iter(train_iter)
batch = next(it) 
print("Size of text batch [max bptt length, batch size]", batch.text.shape)
example = batch.text.transpose('batch', 'seqlen')._tensor[1]
print("Second in batch", example)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in example.data]))

Size of text batch [max bptt length, batch size] OrderedDict([('seqlen', 32), ('batch', 10)])
Second in batch tensor([  72,    5,   28,  247,   61,   12,  216,    5,    0, 1847,   10,    4,
          72,  547,    3, 6506,  163,    7,  105,  479,   38,   31,  295, 4901,
          13,    4,   49,    3,    0,    0,   25, 2471])
Converted back to string:  shares of its common stock for each of <unk> deposit 's N shares outstanding <eos> liberty national a bank holding company has assets exceeding $ N billion <eos> <unk> <unk> was appointed


The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [26]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text.transpose('batch', 'seqlen')._tensor[1].data]))

Converted back to string:  president and chief executive officer of this financially troubled department store chain effective nov. N succeeding frank robertson who is retiring early <eos> mr. <unk> was previously president and chief operating officer


The batch object also contains the targets for the given batch in the field `target`. The target is simply the text offset by one.

In [114]:
text = [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   4,
         73, 394,  34,   0]

In [116]:
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in text]))

Converted back to string:  <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <eos> <unk> <unk> N years old will <unk>


## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/harvard-cs287-s19-hw2/

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 10 possible next word candidates

In [28]:
!head input.txt

but while the new york stock exchange did n't fall ___
some circuit breakers installed after the october N crash failed ___
the N stock specialist firms on the big board floor ___
big investment banks refused to step up to the plate ___
heavy selling of standard & poor 's 500-stock index futures ___
seven big board stocks ual amr bankamerica walt disney capital ___
once again the specialists were not able to handle the ___
<unk> james <unk> chairman of specialists henderson brothers inc. it ___
when the dollar is in a <unk> even central banks ___
speculators are calling for a degree of liquidity that is ___


As a sample Kaggle submission, let us build a simple unigram model.  

In [65]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.values.contiguous().view(-1).tolist())


In [68]:
count

Counter({9971: 1,
         2: 50770,
         9972: 1,
         99: 885,
         9973: 1,
         90: 996,
         9975: 1,
         2255: 42,
         9976: 1,
         9977: 1,
         313: 331,
         9981: 1,
         1642: 62,
         9982: 1,
         5: 24400,
         9983: 1,
         1064: 102,
         9984: 1,
         9: 17474,
         9985: 1,
         714: 151,
         9987: 1,
         9988: 1,
         265: 371,
         9989: 1,
         821: 132,
         9990: 1,
         3: 42068,
         9992: 1,
         9782: 5,
         9993: 1,
         2003: 49,
         9994: 1,
         547: 198,
         9995: 1,
         35: 3245,
         9996: 1,
         2952: 30,
         9997: 1,
         18: 4915,
         9998: 1,
         13: 7541,
         9999: 1,
         4: 32481,
         10000: 1,
         49: 1881,
         156: 595,
         9257: 6,
         0: 45020,
         121: 740,
         1188: 88,
         73: 1241,
         363: 289,
         394: 268,


In [69]:
count[TEXT.vocab.stoi["<eos>"]] = 0


In [70]:
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]


In [71]:
predictions

['the',
 '<unk>',
 'N',
 'of',
 'to',
 'a',
 'in',
 'and',
 "'s",
 'that',
 'for',
 '$',
 'is',
 'it',
 'said',
 'on',
 'by',
 'at',
 'as',
 'from']

In [ ]:
with open("sample.txt", "w") as fout:
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository: https://github.com/harvard-ml-courses/nlp-template

In [31]:
!head sample.txt

id,word
1,the <unk> N of to a in and 's that for $ is it said on by at as from
2,the <unk> N of to a in and 's that for $ is it said on by at as from
3,the <unk> N of to a in and 's that for $ is it said on by at as from
4,the <unk> N of to a in and 's that for $ is it said on by at as from
5,the <unk> N of to a in and 's that for $ is it said on by at as from
6,the <unk> N of to a in and 's that for $ is it said on by at as from
7,the <unk> N of to a in and 's that for $ is it said on by at as from
8,the <unk> N of to a in and 's that for $ is it said on by at as from
9,the <unk> N of to a in and 's that for $ is it said on by at as from


In [32]:
n_words = TEXT.vocab.__len__()

In [33]:
from torch.distributions import dirichlet

In [34]:
# get word counts in training dataset to determine prior
word_counts = torch.zeros([n_words,1],dtype=torch.float32)

for batch in train_iter:
  word_counts[batch.text.values,0]+=1



In [35]:
marginal  = word_counts.squeeze()

In [36]:
conditional_count_1 = torch.zeros([n_words,n_words],dtype=torch.float32)


In [37]:
for batch in train_iter:
  for c in range(29):
    conditional_count_1[batch.text.values[c],batch.text.values[c+1]]+=1

In [0]:
# conditional_count_1 0 = y-1, 1 = y